In [ ]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [ ]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2[df2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [ ]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [ ]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [ ]:
hankel = Hankel()
lag = 60
stride = 0.5

In [ ]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [ ]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
Y_test = test_combined.loc[:,'ATT_FLAG']

In [ ]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [ ]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual,y_predicted))
    sensor_models.append(model)    

In [ ]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual,y_predicted))

Accuracy  0.8117647058823529
Precision  0.8
Recall  0.8695652173913043
F1-score  0.8333333333333333
False Positive Rate  0.2564102564102564


In [ ]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))
print("fpr -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fpr).mean(), np.median(np.asarray(fpr)),np.asarray(fpr).min(), np.asarray(fpr).max()))

Accuracy -  Mean: 0.5034199726402189 Median : 0.5058823529411764 Min : 0.43529411764705883 Max : 0.611764705882353
precision - Mean: 0.6886489479512735 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.08948432760364003 Median : 0.08695652173913043 Min : 0.0 Max : 0.2826086956521739
f1 -        Mean: 0.15405318860972586 Median : 0.16 Min : 0.0 Max : 0.4406779661016949
fpr -        Mean: 0.00834824090638044 Median : 0.0 Min : 0.0 Max : 0.10256410256410256


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [ ]:
cluster_accuracy_1 = []
cluster_precision_1 = []
cluster_recall_1 = []
cluster_fscore_1 = []
cluster_fpr_1 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)  
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    cluster_accuracy_1.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_1.append(precision_score(y_actual,y_predicted))
    cluster_recall_1.append(recall_score(y_actual,y_predicted))
    cluster_fscore_1.append(f1_score(y_actual,y_predicted))
    cluster_fpr_1.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_1[-1])
    print("Precision ",cluster_precision_1[-1])
    print("Recall ",cluster_recall_1[-1])
    print("F1-score ",cluster_fscore_1[-1])
    print("False Positive Rate ",cluster_fpr_1[-1],"/n") 

------Number of Clusters:  2 -----------
Accuracy  0.788235294117647
Precision  0.8043478260869565
Recall  0.8043478260869565
F1-score  0.8043478260869565
False Positive Rate  0.23076923076923078 /n
------Number of Clusters:  3 -----------
Accuracy  0.7294117647058823
Precision  0.7346938775510204
Recall  0.782608695652174
F1-score  0.7578947368421053
False Positive Rate  0.3333333333333333 /n
------Number of Clusters:  4 -----------
Accuracy  0.6941176470588235
Precision  0.6724137931034483
Recall  0.8478260869565217
F1-score  0.75
False Positive Rate  0.48717948717948717 /n
------Number of Clusters:  5 -----------
Accuracy  0.7294117647058823
Precision  0.6949152542372882
Recall  0.8913043478260869
F1-score  0.780952380952381
False Positive Rate  0.46153846153846156 /n
------Number of Clusters:  6 -----------
Accuracy  0.7176470588235294
Precision  0.6774193548387096
Recall  0.9130434782608695
F1-score  0.7777777777777777
False Positive Rate  0.5128205128205128 /n
------Number of Clu

In [ ]:
print("Accuracy Scores: ",cluster_accuracy_1)
print("Precision Scores: ",cluster_precision_1)
print("Recall Scores: ",cluster_recall_1)
print("F1 Scores: ",cluster_fscore_1)
print("False Positive Rates: ",cluster_fpr_1)

Accuracy Scores:  [0.788235294117647, 0.7294117647058823, 0.6941176470588235, 0.7294117647058823, 0.7176470588235294, 0.6823529411764706]
Precision Scores:  [0.8043478260869565, 0.7346938775510204, 0.6724137931034483, 0.6949152542372882, 0.6774193548387096, 0.6507936507936508]
Recall Scores:  [0.8043478260869565, 0.782608695652174, 0.8478260869565217, 0.8913043478260869, 0.9130434782608695, 0.8913043478260869]
F1 Scores:  [0.8043478260869565, 0.7578947368421053, 0.75, 0.780952380952381, 0.7777777777777777, 0.7522935779816514]
False Positive Rates:  [0.23076923076923078, 0.3333333333333333, 0.48717948717948717, 0.46153846153846156, 0.5128205128205128, 0.5641025641025641]


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [ ]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [ ]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [ ]:
cluster_accuracy_2 = []
cluster_precision_2 = []
cluster_recall_2 = []
cluster_fscore_2 = []
cluster_fpr_2 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_2.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_2.append(precision_score(y_actual,y_predicted))
    cluster_recall_2.append(recall_score(y_actual,y_predicted))
    cluster_fscore_2.append(f1_score(y_actual,y_predicted))
    cluster_fpr_2.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_2[-1])
    print("Precision ",cluster_precision_2[-1])
    print("Recall ",cluster_recall_2[-1])
    print("F1-score ",cluster_fscore_2[-1])
    print("False Positive Rate ",cluster_fpr_2[-1],"/n") 

------Number of Clusters:  2 -----------
Accuracy  0.7205882352941176
Precision  0.631578947368421
Recall  0.8275862068965517
F1-score  0.716417910447761
False Positive Rate  0.358974358974359 /n
------Number of Clusters:  3 -----------
Accuracy  0.6470588235294118
Precision  0.5714285714285714
Recall  0.6896551724137931
F1-score  0.625
False Positive Rate  0.38461538461538464 /n
------Number of Clusters:  4 -----------
Accuracy  0.6617647058823529
Precision  0.575
Recall  0.7931034482758621
F1-score  0.6666666666666666
False Positive Rate  0.4358974358974359 /n
------Number of Clusters:  5 -----------
Accuracy  0.6470588235294118
Precision  0.5555555555555556
Recall  0.8620689655172413
F1-score  0.6756756756756757
False Positive Rate  0.5128205128205128 /n
------Number of Clusters:  6 -----------
Accuracy  0.6176470588235294
Precision  0.5306122448979592
Recall  0.896551724137931
F1-score  0.6666666666666666
False Positive Rate  0.5897435897435898 /n
------Number of Clusters:  7 -----

In [ ]:
print("Accuracy Scores: ",cluster_accuracy_2)
print("Precision Scores: ",cluster_precision_2)
print("Recall Scores: ",cluster_recall_2)
print("F1 Scores: ",cluster_fscore_2)
print("False Positive Rates: ",cluster_fpr_2)

Accuracy Scores:  [0.7205882352941176, 0.6470588235294118, 0.6617647058823529, 0.6470588235294118, 0.6176470588235294, 0.6617647058823529]
Precision Scores:  [0.631578947368421, 0.5714285714285714, 0.575, 0.5555555555555556, 0.5306122448979592, 0.56]
Recall Scores:  [0.8275862068965517, 0.6896551724137931, 0.7931034482758621, 0.8620689655172413, 0.896551724137931, 0.9655172413793104]
F1 Scores:  [0.716417910447761, 0.625, 0.6666666666666666, 0.6756756756756757, 0.6666666666666666, 0.7088607594936709]
False Positive Rates:  [0.358974358974359, 0.38461538461538464, 0.4358974358974359, 0.5128205128205128, 0.5897435897435898, 0.5641025641025641]


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [ ]:
cluster_accuracy_3 = []
cluster_precision_3 = []
cluster_recall_3 = []
cluster_fscore_3 = []
cluster_fpr_3 = []
for k in range(2,10):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_3.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_3.append(precision_score(y_actual,y_predicted))
    cluster_recall_3.append(recall_score(y_actual,y_predicted))
    cluster_fscore_3.append(f1_score(y_actual,y_predicted))
    cluster_fpr_3.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_3[-1])
    print("Precision ",cluster_precision_3[-1])
    print("Recall ",cluster_recall_3[-1])
    print("F1-score ",cluster_fscore_3[-1])
    print("False Positive Rate ",cluster_fpr_3[-1],"/n") 

In [ ]:
print("Accuracy Scores: ",cluster_accuracy_3)
print("Precision Scores: ",cluster_precision_3)
print("Recall Scores: ",cluster_recall_3)
print("F1 Scores: ",cluster_fscore_3)
print("False Positive Rates: ",cluster_fpr_3)

Accuracy Scores:  [0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941]
Precision Scores:  [0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941, 0.4264705882352941]
Recall Scores:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
F1 Scores:  [0.5979381443298969, 0.5979381443298969, 0.5979381443298969, 0.5979381443298969, 0.5979381443298969, 0.5979381443298969, 0.5979381443298969, 0.5979381443298969]
False Positive Rates:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
